In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os

<h2> 1. Creating an engine to access the database

In [3]:
#originaly at http://potacho.com/files/ironhack/Albrodmar.db
engine = create_engine('sqlite:///../data/raw/Albrodmar.db')


'/home/alberto/Ironhack/projects/Ironhack-Module-1-Project---Pipeline/notebooks'

<h2>2. Opening all the tables on Pandas

In [4]:
df_business = pd.read_sql_query("SELECT * FROM business_info", engine)

In [5]:
df_personal = pd.read_sql_query("SELECT * FROM personal_info", engine)

In [6]:
df_rank = pd.read_sql_query('SELECT * FROM rank_info', engine)


<h4> All the df have a common column 'id' but df_rank has more lines thah the others df. Will 'outer' join the three base on 'id'


<h2 style="color:green;"> >>>> REGENERATE DATAFRAME <<<<

In [7]:
df_temp = pd.merge(df_business, df_personal, on = 'id', how = 'outer')
df_merged = pd.merge(df_temp, df_rank, on = 'id', how = 'outer')
df_merged.head(2)

,id,Unnamed: 0_x,Source,worth,worthChange,realTimeWorth,realTimePosition,lastName,age,Unnamed: 0_y,gender,country,image,name,position,Unnamed: 0
0,5390,52.0,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,None,1.0,bEZOS,54 years old,52.0,M,United States,https://specials-images.forbesimg.com/imageser...,jEFF BEzos,1.0,52
1,1675,53.0,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,None,2.0,GATEs,62 years old,53.0,None,None,https://specials-images.forbesimg.com/imageser...,BILL GAtes,2.0,53


<h2> 3. Start Data Wrangling

In [20]:
df_merged.isnull().sum()

id                     0
Unnamed: 0_x          52
Source                52
worth                 52
worthChange           52
realTimeWorth       2260
realTimePosition      52
lastName              52
age                  117
Unnamed: 0_y          52
gender                65
country               52
image                 52
name                   0
position              52
Unnamed: 0             0
dtype: int64

<h4>3.1 Drop columns: <ul><b>realTimeWorth</b> because is full of nulls and column</ul> <ul><b>lastName</b> because is useless having column <b>name</b></ul><ul><b>'Unnamed: 0_y'</b></ul>
    <ul><b>'Unnamed:0_x'</b></ul><ul><b>'Unnamed: 0'</b> (got same values and I don't know their meaning. There are not repeated values. Seem like they are ID columns only).</ul><ul><b>position</b></ul><ul><b>worthChange</b></ul><ul><b>realTimePosition</b></ul>

In [21]:
df_merged = df_merged.drop(['realTimeWorth', 'lastName', 'Unnamed: 0_x', 'Unnamed: 0_y', 'Unnamed: 0', 'position', 'worthChange', 'realTimePosition'], axis = 1)

In [22]:
df_merged.head()

,id,Source,worth,age,gender,country,image,name
0,5390,Technology ==> Amazon,112.0 BUSD,54 years old,M,United States,https://specials-images.forbesimg.com/imageser...,jEFF BEzos
1,1675,Technology ==> Microsoft,90.0 BUSD,62 years old,None,None,https://specials-images.forbesimg.com/imageser...,BILL GAtes
2,2361,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,87 years old,None,None,https://specials-images.forbesimg.com/imageser...,warREN BUFfett
3,2340,Fashion & Retail ==> LVMH,72.0 BUSD,69 years old,None,None,https://specials-images.forbesimg.com/imageser...,bERNARD ARNAUlt
4,6891,Technology ==> Facebook,71.0 BUSD,1985,None,None,https://specials-images.forbesimg.com/imageser...,marK ZUCKERberg


<h4>3.2 Separete column 'Source' in column "Industry" and "Company"

In [23]:
df_merged[['industry','company_name']] = df_merged.Source.str.split(' ==>',expand = True)

#drop old column Source
df_merged = df_merged.drop(['Source'], axis = 1)

<h4>3.3 Wrangling column <b>'name'</b>

<h5>3.3.1 Capitalizing first letters only on column 'name'

In [24]:
df_merged['name'] = df_merged['name'].str.title()

<h4>3.4 Wrangling column <b>'age'</b>

<h5>3.4.1 Cleaning the 'age' column removing ' years old'

In [25]:
df_merged['age'] = df_merged.age.str.replace(' years old','')

<h5>3.4.2 Convert data in column 'age' to float

In [26]:
df_merged['age'] = df_merged['age'].apply(pd.to_numeric)

<h5>3.4.3 Changing 'ages' with a year of birth to years old 

In [32]:
df_merged['age'] = df_merged['age'].apply(lambda x: 2018 - x if x > 150 else x)

<h4>3.5 Wrangling column <b>'worth'</b>

<h5>3.5.1 Cleaning the 'worth' column removing 'BUSD'

In [27]:
df_merged['worth'] = df_merged.worth.str.replace('BUSD','')

<h5>3.5.2 Changing name of column '<b>worth</b>'

In [29]:
df_merged.rename(columns = {'worth':'worth in Billion USD', 'country':'Country'}, inplace = True)

<h4>3.5.3 Convert data in column 'worth in Billion USD' to float

In [30]:
df_merged['worth in Billion USD'] = df_merged['worth in Billion USD'].apply(pd.to_numeric)

In [31]:
df_merged.head(2)

,id,worth in Billion USD,age,gender,Country,image,name,industry,company_name
0,5390,112.0,54.0,M,United States,https://specials-images.forbesimg.com/imageser...,Jeff Bezos,Technology,Amazon
1,1675,90.0,62.0,None,None,https://specials-images.forbesimg.com/imageser...,Bill Gates,Technology,Microsoft


<h4>3.6 Wranglin column <b>'gender'</b>

<h5>3.6.1 Changing 'gender' values to be only Male or Female

In [34]:
m_filter = df_merged['gender'] == 'M'
df_merged.loc[m_filter,'gender'] = df_merged.loc[m_filter,'gender'].replace('M','Male')

In [42]:
f_filter = df_merged['gender'] == 'F'
df_merged.loc[f_filter,'gender'] = df_merged.loc[f_filter,'gender'].replace('F','Female')

In [43]:
df_merged['gender'].value_counts()

Male      1467
None       549
Female     179
Name: gender, dtype: int64

<h4>3.7 Wrangling column <b>'Country'</b>

In [44]:
df_merged[df_merged['Country'].isnull()].count()

id                      52
worth in Billion USD     0
age                      0
gender                   0
Country                  0
image                    0
name                    52
industry                 0
company_name             0
dtype: int64

<h4>3.8 Decide to drop all registers with 'country' null because there have no values in the rest of the registries

In [45]:
df = df_merged[pd.notnull(df_merged['Country'])]
df.head()
df[df['Country'] == 'None'].name #shows the ones that values (string) None on 'Country'

1             Bill Gates
2         Warren Buffett
3        Bernard Arnault
4        Mark Zuckerberg
6       Carlos Slim Helu
              ...       
2198         Sandy Weill
2199       Wen Pengcheng
2200         Wu Chung-Yi
2206            Zhuo Jun
2207         Olgun Zorlu
Name: name, Length: 1406, dtype: object

<h4> Create csv file with the df

In [46]:
df.to_csv('../data/processed/billionaires.csv', index = False)

In [47]:
df

,id,worth in Billion USD,age,gender,Country,image,name,industry,company_name
0,5390,112.0,54.0,Male,United States,https://specials-images.forbesimg.com/imageser...,Jeff Bezos,Technology,Amazon
1,1675,90.0,62.0,None,None,https://specials-images.forbesimg.com/imageser...,Bill Gates,Technology,Microsoft
2,2361,84.0,87.0,None,None,https://specials-images.forbesimg.com/imageser...,Warren Buffett,Finance and Investments,Berkshire Hathaway
3,2340,72.0,69.0,None,None,https://specials-images.forbesimg.com/imageser...,Bernard Arnault,Fashion & Retail,LVMH
4,6891,71.0,33.0,None,None,https://specials-images.forbesimg.com/imageser...,Mark Zuckerberg,Technology,Facebook
...,...,...,...,...,...,...,...,...,...
2203,2797,1.0,51.0,Male,China,https://specials-images.forbesimg.com/imageser...,Zhao Xiaoqiang,Fashion & Retail,"fashion, entertainment"
2204,7549,1.0,55.0,Male,People's Republic of China,https://specials-images.forbesimg.com/imageser...,Zhou Liangzhang,Manufacturing,electrical equipment
2205,6404,1.0,51.0,Male,People's Republic of China,https://specials-images.forbesimg.com/imageser...,Zhu Xingming,Manufacturing,electrical equipment
2206,3849,1.0,52.0,Female,None,https://specials-images.forbesimg.com/imageser...,Zhuo Jun,Manufacturing,printed circuit boards
